# Basic Aerodynamics

In [3]:
import pint
u = pint.UnitRegistry()

In aerodynamics, we work with non-dimensional coefficients to that help us compare airfoil sections. Here are the basic force definitions used:

\begin{equation}
L = \frac{1}{2}\rho u^2 C_l S
\end{equation}

\begin{equation}
D = \frac{1}{2}\rho u^2 C_d S
\end{equation}

\begin{equation}
M = \frac{1}{2}\rho u^2 C_n S c
\end{equation}

In these equations, $\rho$ is the density of the air, $u$ is the forward velocity, $S$ is a reference area, usually the projected area of the surface, and $c$ is a reference length, usually the chord of the surface.

Based on McLean's development, we assume that model is trimmed so it is flying with these aerodynamic coefficient values:

- $C_{l,w}$ = 1.0 - wing lift coefficient
- $C_{m,w}$ = -0.10 - wing pitching moment
- $C_{m,s}$ = 0.05 - stab pitching moment

In order to maintain that level flight, we need to balance all of the forces acting on the model. However, we are missing three coefficients: The drag coefficients for the wing and stab, and the lift coefficient for the stab. (Notice we have not mentioned the airfoil in any of this so far!)

## Simplified Airfoil Theory

Long before computers first appeared, aeronautical research involved watching vehicles fly and thinking about what was going on in the air around that vehicle. They then tried to come up with mathematical approximations that seemed to agree with their observations. This thinking led to two theories that we will use here: *Thin Airfoil Theory* developed by Munk and Glauert in the 1920's {cite}`glauert` and *Lifting Line Theory* developed by Lanchester and Prandtl {cite}`prandtl`shortly after the Wright Brothers first flew. Their work tried to account for the  vortexes observed at the tips of flying surfaces.

### Thin Airfoil Theory

We do not need to explore this theory here, but the key development we need is this approximation for the lift coefficient:

\begin{equation}
C_l = c_{l,0} + 2\pi\alpha
\end{equation}

Here, $\alpha$ is the angle of attack relative to the forward velocity, and $c_{l,0}$ is the lift coefficient at $\alpha = 0$. For the cambered airfoils we use, this is non-zero, unless your airfoil is flat!

### Lifting-Line Theory

If you try to use the theory of wing sections developed using Thin Airfoil Theory to a 3-dimensional wing, you run into a problem. The actual lift generated across the span is not constant. It drops off as you reach the tips. In an attempt to compensate for this, Lanchester and Prandtl came up with an adjustment to the calculations that seemed to account for variations in aspect ratios that were observed.

The end result of this theory is the introduction of an *induced drag* component:

\begin{equation}
C_{d,i} = \frac{C_l^2}{\pi A_R e}
\end{equation}

Here, $AR$ is the aspect ratio of the wing, and $e$ is an *efficiency factor*.


This *induced drag* is only part of he total drag produced by a flying wing. There is also a surface drag caused by the friction of the air moving over the surface, for instance, a *profile drag* caused by pushing the surface through the air.

The profile drag can be approximated by:

\begin{equation}
C_p = 
\end{equation}

The lift and drag forces will be calculated using the definitions of the lift and drag coefficients:
    
\begin{equation}
C_l = \frac{2L}{\rho u^2 S}
\end{equation}

or



IN these equations, **L** and **D** are the lift forces, **S** is the surface area of the wing or stab, $\rho$ is the air density and **u** is the flight velocity. 

Similarly:


Formally, we should also include the moment coefficient, which accounts for the center of pressure not falling exactly at the aerodynamic center



Where **c** is the mean chord.

For testing purposes, we will use these values for density and velocity:



In [4]:
arms = {
    'thrust': (-2.5 * u.inch, -0.125*u.inch),
    'weight': (0*u.inch,0*u.inch),
    'wing_ac': (-1*u.inch,1*u.inch),
    'stab_ac': (4*u.inch,-1*u.inch),
}

In [5]:
forces = {
    'thrust': 0.05*u.grams,
    'weight': 2.06*u.grams,
    'lift_wing': 1.5*u.grams,
    'drag_wing': 0.03*u.grams,
    'lift_stab': -0.4*u.grams,
    'drag_stab': 0.015*u.grams
}

We can now write a simple function that determines the moment around the **CG** for given values of the various terms. To simplify this function, we will create a Python dictionary holding the values.

It should be apparent that all six force vectors need to be considered, since none of them (except weight) align with the center of gravity. 

In [6]:
def moment(a,f, debug=False):
    
    # thrust moment
    ta = a['thrust'][1] # y 
    tf = f['thrust']
    tm = ta * tf
    if debug:
        print("Thrust moment:",tm)
    # weight  - no moment, but we will calculate it anyway
    wa = a['weight'][0]
    wf = f['weight']
    wm = wa * wf
    if debug:
        print("Weight moment:",wm)
    
    # wing lift moment
    wla = a['wing_ac'][0]
    wlf = f['lift_wing']
    wlm = wla * wlf
    if debug:
        print("Wing lift moment:",wlm)
    
    # wing drag moment
    wda = a['wing_ac'][1]
    wdf = f['drag_wing']
    wdm = wda * wdf
    if debug:
        print("Wing drag moment:",wdm)    
    
    # stab lift moment
    sla = a['stab_ac'][0]
    slf = f['lift_stab']
    slm = sla * slf
    if debug:
        print("Stab lift moment:",slm)    
    
    # stab drag moment
    sda = a['stab_ac'][1]
    sdf = f['drag_stab']
    sdm = sda * sdf 
    if debug:
        print("Stab drag moment:",sdm)
        
    total_moment = tm + wm + wlm + wdm + slm + sdm 
    print('total moment =',total_moment)


In [7]:
moment(arms, forces, debug=True)

Thrust moment: -0.00625 gram * inch
Weight moment: 0.0 gram * inch
Wing lift moment: -1.5 gram * inch
Wing drag moment: 0.03 gram * inch
Stab lift moment: -1.6 gram * inch
Stab drag moment: -0.015 gram * inch
total moment = -3.09125 gram * inch


By convention, a positive moment pitched the node up. So this one is going to fly about like many of my  first flights!

For stable level flight, the total moment needs to be zero. Obviously, I threw in random values here, and our goal is to produce all of these numbers from our design and a little help from aerodynamics theory.

The nice thing about using code to help us make these calculations is that we can run parametric tests and see results quickly, We will use that fact later.

let's see what we have now:

We estimated the mean chord values by dividing the projected area by the projected span. The location of the mean aerodynamic centers for both surfaces is set at the quarter chord point. We will locate it from the trailing edges since both are straight on the plans.



In [8]:
flight = {
    'density': 0.00231683 *u.slugs/u.ft**3,
    'dynamic_viscosity': 0.0000180338 * u.pascal * u.second,
    'velocity': 2.0 * u.mph
}
flight['kinematic_viscosity'] = flight['dynamic_viscosity']/flight['density']
flight['q'] = 0.5* (flight['density'] * flight['velocity'] ** 2)
flight['wing_re'] = flight['density'] * flight['velocity'] * wing['mean_chord'] / flight['dynamic_viscosity']
flight['stab_re'] = flight['density'] * flight['velocity'] * stab['mean_chord'] / flight['dynamic_viscosity']

dict_print(flight)

NameError: name 'wing' is not defined

In [102]:
wing_re = flight['wing_re'].to_base_units()
wing_re

2917.575722064471 <Unit('dimensionless')>

In [104]:
stab_re = flight['stab_re'].to_base_units()
stab_re

2067.5036243705945 <Unit('dimensionless')>

Getting airfoil data for these Reynolds Numbers is not easy. McLean arbitrarily used a $C_lw$ of 1.0, and calculated the required $C_ls* to balance moments. He used pitching moment coefficient values of -0.10 and -0.05 for the wing and stab.He did not include any other forces. 

Rather than just use these numbers, we will use the most basic theories of aerodynamics to estimate the coefficients. 

*Thin Airfoil Theory*, is often used as a first approximation for calculating lift and pitching moment data. If we use *Prandtl's Lifting Line Theory* we can get an approximation for the induced drag, and profile drag which together as the major sources of drag at low speeds.

we can estimate the $C_d$ values by examining typical lift/drag curves. 
